In [4]:
import xarray as xr
import xroms
import numpy as np
import matplotlib.pyplot as plt

Testing to check out data

In [28]:
base_url_turb_avg_02 = 'https://thredds.met.no/thredds/dodsC/havvind/ocean/turbines/EXP-02/norkyst_avg_'
base_url_turb_avg_03 = 'https://thredds.met.no/thredds/dodsC/havvind/ocean/turbines/EXP-03/norkyst_avg_'
base_url_turb_avg_04 = 'https://thredds.met.no/thredds/dodsC/havvind/ocean/turbines/EXP-04/norkyst_avg_'
base_url_turb_avg_05 = 'https://thredds.met.no/thredds/dodsC/havvind/ocean/turbines/EXP-05/norkyst_avg_'
base_url_turb_avg_06 = 'https://thredds.met.no/thredds/dodsC/havvind/ocean/turbines/EXP-06/norkyst_avg_'

In [37]:
def open_experiment(base_url, num_of_days):
    paths = []
    for i in range(1, num_of_days+1):
        file_number = f'{i:04}'
        file_path = f'{base_url}{file_number}.nc'
        paths.append(file_path)

    ds = xroms.open_mfnetcdf(paths)

    return ds

In [38]:
turb_avg_02 = open_experiment(base_url_turb_avg_02, 27)
turb_avg_03 = open_experiment(base_url_turb_avg_03, 31)
turb_avg_04 = open_experiment(base_url_turb_avg_04, 30)
turb_avg_05 = open_experiment(base_url_turb_avg_05, 31)
turb_avg_06 = open_experiment(base_url_turb_avg_06, 30)

In [39]:
turb_avg_02

<xarray.Dataset>
Dimensions:         (tracer: 2, boundary: 4, s_rho: 40, s_w: 41, eta_rho: 1148,
                     xi_rho: 2747, xi_u: 2746, eta_v: 1147, ocean_time: 27)
Coordinates: (12/23)
  * s_rho           (s_rho) float64 -0.9799 -0.9403 ... -0.01529 -0.004904
  * s_w             (s_w) float64 -1.0 -0.96 -0.9208 ... -0.02077 -0.01 0.0
    lon_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(1148, 2747), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(1148, 2747), meta=np.ndarray>
    lon_u           (eta_rho, xi_u) float64 dask.array<chunksize=(1148, 2746), meta=np.ndarray>
    lat_u           (eta_rho, xi_u) float64 dask.array<chunksize=(1148, 2746), meta=np.ndarray>
    ...              ...
    z_w_v           (ocean_time, s_w, eta_v, xi_rho) float64 dask.array<chunksize=(1, 41, 1147, 2747), meta=np.ndarray>
    z_w_psi         (ocean_time, s_w, eta_v, xi_u) float64 dask.array<chunksize=(1, 41, 1147, 2746), meta=np.ndarray>
    z_rho           (ocean_time, s_rho, eta_rho, xi_rho) float64 dask.array<chunksize=(1, 40, 1148, 2747), meta=np.ndarray>
    z_rho_u         (ocean_time, s_rho, eta_rho, xi_u) float64 dask.array<chunksize=(1, 40, 1148, 2746), meta=np.ndarray>
    z_rho_v         (ocean_time, s_rho, eta_v, xi_rho) float64 dask.array<chunksize=(1, 40, 1147, 2747), meta=np.ndarray>
    z_rho_psi       (ocean_time, s_rho, eta_v, xi_u) float64 dask.array<chunksize=(1, 40, 1147, 2746), meta=np.ndarray>
Dimensions without coordinates: tracer, boundary
Data variables: (12/123)
    ntimes          int32 ...
    ndtfast         int32 ...
    dt              float64 ...
    dtfast          float64 ...
    dstart          datetime64[ns] ...
    shuffle         int32 ...
    ...              ...
    dz_w_u          (ocean_time, s_w, eta_rho, xi_u) float64 dask.array<chunksize=(1, 41, 1148, 2746), meta=np.ndarray>
    dz_v            (ocean_time, s_rho, eta_v, xi_rho) float64 dask.array<chunksize=(1, 40, 1147, 2747), meta=np.ndarray>
    dz_w_v          (ocean_time, s_w, eta_v, xi_rho) float64 dask.array<chunksize=(1, 41, 1147, 2747), meta=np.ndarray>
    dz_psi          (ocean_time, s_rho, eta_v, xi_u) float64 dask.array<chunksize=(1, 40, 1147, 2746), meta=np.ndarray>
    dz_w_psi        (ocean_time, s_w, eta_v, xi_u) float64 dask.array<chunksize=(1, 41, 1147, 2746), meta=np.ndarray>
    dA              (eta_rho, xi_rho) float64 dask.array<chunksize=(1148, 2747), meta=np.ndarray>
Attributes: (12/34)
    file:                            /nobackup/forsk/sm_kaich/havvind/run/nor...
    format:                          netCDF-4/HDF5 file
    Conventions:                     CF-1.4, SGRID-0.3
    type:                            ROMS/TOMS nonlinear model averages file
    title:                           Norkyst V3 forecast model (MET Norway/IMR)
    var_info:                        /home/sm_kaich/havvind/norkyst_varinfo_s...
    ...                              ...
    compiler_flags:                  -O3 -mcmodel=large
    tiling:                          032x016
    history:                         ROMS/TOMS, Version 3.9, Wednesday - Marc...
    ana_file:                        ROMS/Functionals/ana_btflux.h
    CPP_options:                     NORKYST, ADD_FSOBC, ADD_M2OBC, ANA_BSFLU...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time